In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import glob
import os
%matplotlib inline

## Model

In [2]:
def conf_mat(pred,truth):
    res = [0,0,0,0]
    a = 0
    
    for i in range(len(truth)):
        if truth[i] == 1:
            if truth[i] == pred[i]:
                a = 0
            else:
                a = 2
                
        else:
            if truth[i] == pred[i]:
                a = 1
            else:
                a = 3
                
        res[a] = res[a] + 1
     
    print(res)
    return res

## Multi-dataset Model

In [7]:
input_dir = './../../train/KPI'

summary = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])

for fname in os.listdir(input_dir):
    df  = pd.read_csv(os.path.join(input_dir, fname), index_col='timestamp')
    kpi_name = df['KPI ID'].values[0]
    print(kpi_name)
    df = df.drop(['KPI ID'], axis=1)
    
    # Normalize Values
    normalized_df=(df-df.min())/(df.max()-df.min())
    normalized_df = normalized_df.astype({'label': 'int64'})
    
    # Split to Train and Test
    train_set, test_set= np.split(normalized_df, [int(.75 *len(normalized_df))])
    
    # Format Train and Test
    X = np.array(train_set['value']).reshape(-1, 1)
    y = np.array(train_set['label'])
    x_test = np.array(test_set['value']).reshape(-1,1)
    y_test = np.array(test_set['label'])
    
    
    # Check Valid Train Dataset
    if len(np.unique(y)) > 1:

        # Train Model
        model = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                              hidden_layer_sizes=(20,2), random_state=1)
        model.fit(X,y)

        # Make Predictions
        predictions = model.predict(x_test)

        # Compute Confusion Matrix
        cf = conf_mat(predictions,y_test) 

        # F1-Score
        prec = 0
        rec = 0
        f1 = 0
        if (cf[0] + cf[2]) != 0:
            prec = cf[0] / (cf[0] + cf[2])
        if (cf[0] + cf[3]) != 0:
            rec  = cf[0] / (cf[0] + cf[3])
        if (prec + rec) != 0:
            f1   = 2 * (prec * rec / (prec+rec))

    #     print(f1_score(predictions,y_test))


        summary = summary.append({'KPI': kpi_name, 
                                   'TP': cf[0],  
                                   'TN': cf[1], 
                                   'FP': cf[2], 
                                   'FN': cf[3], 
                                   'PRECISION': prec, 
                                   'RECALL': rec,     
                                   'F1_SCORE': f1 }, ignore_index=True)
        
    else:
        summary = summary.append({'KPI': kpi_name, 
                                   'TP': None,  
                                   'TN': None, 
                                   'FP': None, 
                                   'FN': None, 
                                   'PRECISION': None, 
                                   'RECALL': None,     
                                   'F1_SCORE': None }, ignore_index=True)
    

a40b1df87e3f1c87
[0, 34376, 106, 0]
b3b2e6d1a791d63a
[0, 2062, 0, 0]
da403e4e3f87c9e0
[123, 31997, 139, 0]
046ec29ddf80d62e
[0, 2196, 0, 0]
18fbb1d5a5dc099d
[15, 32170, 36, 61]
09513ae3e75778a3
[0, 32115, 128, 0]
07927a9a18fa19ae
[7, 2716, 17, 0]
cff6d3c01e6a6bfa
[0, 36619, 289, 0]
54e8a140f6237526
[0, 2062, 0, 0]
c58bfcbacb2822d1
[0, 32120, 47, 0]
7c189dd36f048a6c
[0, 36843, 80, 0]
8bef9af9a922e0b3
[0, 32288, 76, 0]
40e25005ff8992bd
[0, 24904, 160, 0]
76f4550c43334374
[0, 2192, 4, 0]
affb01ca2b4f0b45
[14, 36809, 79, 18]
88cf3a776ba00e7c
[0, 15548, 811, 0]
9bd90500bfd11edb
[0, 32154, 0, 0]
e0770391decc44ce
[0, 34050, 2706, 0]
8a20c229e9860d0c
769894baefea4e9e
[0, 2190, 6, 0]
1c35dbf57f55f5e4
[2687, 28507, 25, 995]
9ee5879409dccef9
[0, 15581, 782, 0]
71595dd7171f4540
[0, 36640, 277, 0]
8c892e5525f3e491
[0, 34067, 2686, 0]
a5bf5d65261d859a
[0, 32160, 10, 0]
02e99bd4f6cfb33f
[731, 30558, 651, 201]


In [8]:
summary.to_csv('MLP_Result1.csv')